In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
pd.options.display.max_columns = None

In [24]:
df = pd.read_csv("../data/raw/transaction_history.csv")
df.info()

/tmp/ipykernel_7919/4132290206.py:1: DtypeWarning: Columns (3,9,10,11,12,16,18,20,24,27,31,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/transaction_history.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168108 entries, 0 to 168107
Data columns (total 37 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 168108 non-null  int64  
 1   stock_code                 168108 non-null  object 
 2   transaction_date           168108 non-null  object 
 3   closing_price              163607 non-null  object 
 4   adjusted_price             163607 non-null  object 
 5   change_hisCost             163607 non-null  object 
 6   matched_volume             163607 non-null  object 
 7   matched_value              163607 non-null  float64
 8   negotiated_volume          163607 non-null  object 
 9   negotiated_value           163607 non-null  object 
 10  opening_price              163607 non-null  object 
 11  highest_price              163607 non-null  object 
 12  lowest_price               163607 non-null  object 
 13  company_name_foriegnBlock  16

Xóa các cột nhiều dữ liệu trống

In [4]:
df.drop(columns=["proprietary_buy_volume", "proprietary_buy_value", 
                 "proprietary_sell_volume", "proprietary_sell_value",
                 "proprietary_net_volume", "proprietary_net_value",
                 "Unnamed: 0"], inplace=True)

In [5]:
df.head()

,stock_code,transaction_date,closing_price,adjusted_price,change_hisCost,matched_volume,matched_value,negotiated_volume,negotiated_value,opening_price,highest_price,lowest_price,company_name_foriegnBlock,change_foriegnBlock,foreign_net_volume,foreign_net_value,foreign_buy_volume,foreign_buy_value,foreign_sell_volume,foreign_sell_value,remaining_room,foreign_ownership,change_orderStat,buy_orders,buy_volume,avg_buy_volume_per_order,sell_orders,sell_volume,avg_sell_volume_per_order,net_volume
0,ABC,01/02/2018,14.8,11.75,-0.6(-3.90 %),"32,900",0.49,0,0.0,15.3,15.3,14.0,CTCP Truyền thông VMG,14.8(-3.90 %),0,0.0,0,0.0,0,0.0,4,49%,14.8 ( -3.9 %),74,"69,900",944.59,49,"48,200",984,"21,700"
1,ABC,01/02/2019,9.2,7.3,0(0.00 %),"4,310",0.04,0,0.0,9.2,9.4,9.2,CTCP Truyền thông VMG,9.2(0.00 %),0,0.0,0,0.0,0,0.0,60,49%,9.2 ( 0 %),16,"15,700",981.25,16,"10,100",631,"5,600"
2,ABC,01/02/2021,15.9,13.87,-0.4(-2.45 %),"10,000",0.16,0,0.0,16.0,17.0,15.9,CTCP Truyền thông VMG,15.9(-2.45 %),0,0.0,0,0.0,0,0.0,0,49%,15.9 ( -2.45 %),63,"41,700",661.9,64,"60,200",941,"-18,500"
3,ABC,01/02/2023,7.2,6.28,0(0.00 %),0,0.00,0,0.0,7.2,7.2,7.2,CTCP Truyền thông VMG,7.2(0.00 %),0,0.0,0,0.0,0,0.0,"9,800,002",0.94%,7.2 ( 0 %),0,0,0,0,0,0,0
4,ABC,01/02/2024,13.2,12.64,-0.2(-1.49 %),811,0.01,0,0.0,13.2,13.2,13.2,CTCP Truyền thông VMG,13.2(-1.49 %),496,0.01,500,0.01,4,0.0,"9,819,898",0%,13.2 ( -1.49 %),21,"18,800",895.24,26,"36,700","1,412","-17,900"


In [6]:
def preprocess_stock_data(df):
    df.replace(["--", "-", "NaN", "N/A", ""], np.nan, inplace=True)
    # Chuyển đổi transaction_date sang kiểu datetime
    df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d/%m/%Y', errors="coerce")
    
    # Loại bỏ dấu phẩy trong các cột số và chuyển kiểu dữ liệu
    numeric_cols = ['closing_price', 'adjusted_price', 'matched_volume', 'matched_value',
                    'negotiated_volume', 'negotiated_value', 'opening_price', 'highest_price',
                    'lowest_price', 'foreign_net_volume', 'foreign_net_value', 'foreign_buy_volume',
                    'foreign_buy_value', 'foreign_sell_volume', 'foreign_sell_value', 'remaining_room',
                    'buy_orders', 'buy_volume', 'avg_buy_volume_per_order', 'sell_orders', 'sell_volume',
                    'avg_sell_volume_per_order', 'net_volume']
    for col in numeric_cols:
        df[col] = df[col].astype(str).str.replace(',', '').astype(float)
    
    df['change_hisCost'] = df['change_hisCost'].str.replace(r'\s+', '', regex=True)
    df['change_foriegnBlock'] = df['change_foriegnBlock'].str.replace(r'\s+', '', regex=True)
    df['change_orderStat'] = df['change_orderStat'].str.replace(r'\s+', '', regex=True)

    # Cải tiến regex để giữ lại dấu âm
    df[['change_hisCost_value', 'change_hisCost_percent']] = df['change_hisCost'].str.extract(r'(-?\d+\.\d+|-?\d+)\((-?\d+\.\d+|-?\d+)%\)')
    df[['change_foriegnBlock_value', 'change_foriegnBlock_percent']] = df['change_foriegnBlock'].str.extract(r'(-?\d+\.\d+|-?\d+)\((-?\d+\.\d+|-?\d+)%\)')
    df[['change_orderStat_value', 'change_orderStat_percent']] = df['change_orderStat'].str.extract(r'(-?\d+\.\d+|-?\d+)\((-?\d+\.\d+|-?\d+)%\)')

    # Chuyển đổi sang kiểu số
    df[['change_hisCost_value', 'change_hisCost_percent',
        'change_foriegnBlock_value', 'change_foriegnBlock_percent',
        'change_orderStat_value', 'change_orderStat_percent']] = df[[
                                                                    'change_hisCost_value', 'change_hisCost_percent',
                                                                    'change_foriegnBlock_value', 'change_foriegnBlock_percent',
                                                                    'change_orderStat_value', 'change_orderStat_percent']].astype(float)
                                                                
    df["foreign_ownership"] = pd.to_numeric(df["foreign_ownership"].str.replace("%", ""), errors="coerce")
    # Xóa các cột gốc đã được tách
    df.drop(columns=['change_hisCost', 'change_foriegnBlock', 'change_orderStat'], inplace=True)
    
    return df


In [7]:
df_cleaned = preprocess_stock_data(df)

In [36]:
df.loc[(df['stock_code'] == "YEG") & df['closing_price'].isna()]

,Unnamed: 0,stock_code,transaction_date,closing_price,adjusted_price,change_hisCost,matched_volume,matched_value,negotiated_volume,negotiated_value,opening_price,highest_price,lowest_price,company_name_foriegnBlock,change_foriegnBlock,foreign_net_volume,foreign_net_value,foreign_buy_volume,foreign_buy_value,foreign_sell_volume,foreign_sell_value,remaining_room,foreign_ownership,change_orderStat,buy_orders,buy_volume,avg_buy_volume_per_order,sell_orders,sell_volume,avg_sell_volume_per_order,net_volume,proprietary_buy_volume,proprietary_buy_value,proprietary_sell_volume,proprietary_sell_value,proprietary_net_volume,proprietary_net_value
166428,166428,YEG,01/01/0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTCP Tập đoàn Yeah1,0(0.00 %),0,0,0,0,0,0.0,"26,131,115",16.46%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166528,166528,YEG,03/02/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTCP Tập đoàn Yeah1,14.85(2.06 %),"-61,970",-0.91,"26,980",0.4,"88,950",1.31,0,0%,0 ( 0 %),"1,635","3,493,500","2,136.7","1,184","3,474,700","2,935","18,800",NaN,NaN,NaN,NaN,NaN,NaN
166980,166980,YEG,11/03/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTCP Tập đoàn Yeah1,16.25(1.88 %),"7,200",0.09,"126,500",2.03,"119,300",1.95,0,0%,16.25 ( 1.88 %),"3,653","8,350,700","2,285.98","2,734","9,392,900","3,436","-1,042,200",NaN,NaN,NaN,NaN,NaN,NaN
167033,167033,YEG,12/03/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTCP Tập đoàn Yeah1,16(-1.54 %),"-26,100",-0.42,700,0.01,"26,800",0.43,0,0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,"20,000",0.32,"-20,000",-0.32
167091,167091,YEG,13/03/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTCP Tập đoàn Yeah1,15.85(-0.94 %),-500,-0.01,500,0.01,"1,000",0.02,0,0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167914,167914,YEG,27/11/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTCP Tập đoàn Yeah1,0(0.00 %),"-53,530",-2.36,10,0,"53,540",2.36,"21,906,737",29.97%,0 ( 0 %),98,"139,410","1,422.55",63,"143,160","2,272","-3,750",NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_cleaned = df_cleaned.sort_values(by=["stock_code", "transaction_date"])

In [12]:
df_cleaned.isna().sum()

stock_code                         0
transaction_date                   1
closing_price                   4509
adjusted_price                 23659
matched_volume                  4501
matched_value                   4501
negotiated_volume               4501
negotiated_value                4501
opening_price                   4501
highest_price                   4501
lowest_price                    4501
company_name_foriegnBlock       1997
foreign_net_volume              1997
foreign_net_value               1997
foreign_buy_volume              1997
foreign_buy_value               1997
foreign_sell_volume             1997
foreign_sell_value              1997
remaining_room                  1997
foreign_ownership               1999
buy_orders                      3622
buy_volume                      3622
avg_buy_volume_per_order        3622
sell_orders                     3622
sell_volume                     3622
avg_sell_volume_per_order       3622
net_volume                      3622
c

In [ ]:
df

In [17]:
df_select = df_cleaned[df_cleaned['transaction_date'] < "2025-01-01" ]

In [19]:
df_select.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165626 entries, 1216 to 168107
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   stock_code                   165626 non-null  object        
 1   transaction_date             165626 non-null  datetime64[ns]
 2   closing_price                161252 non-null  float64       
 3   adjusted_price               144297 non-null  float64       
 4   matched_volume               161252 non-null  float64       
 5   matched_value                161252 non-null  float64       
 6   negotiated_volume            161252 non-null  float64       
 7   negotiated_value             161252 non-null  float64       
 8   opening_price                161252 non-null  float64       
 9   highest_price                161252 non-null  float64       
 10  lowest_price                 161252 non-null  float64       
 11  company_name_foriegnBlock   

In [20]:
df_select.to_csv("../data/processed/transaction_history.csv", index=False)